In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [2]:
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [3]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line)
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [4]:
!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

--2017-05-04 12:32:47--  https://dsr-data.s3.amazonaws.com/flights/flights14.csv
Resolving dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)... 54.231.193.48
Connecting to dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)|54.231.193.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16150465 (15M) [text/csv]
Saving to: ‘flights14.csv’

flights14.csv       100%[=====================>]  15.40M  4.71MB/s   in 3.5s   

2017-05-04 12:32:51 (4.44 MB/s) - ‘flights14.csv’ saved [16150465/16150465]



In [5]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [6]:
flightsDF = sqlContext.createDataFrame(flights)

In [7]:
flightsDF.createOrReplaceTempView("flight")

In [8]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled='0', carrier='AA', day=1, dep_delay=14, dep_time=914, dest='LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin='JFK', tailnum='N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1902, dest='LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin='JFK', tailnum='N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1347, dest='LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin='JFK', tailnum='N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled='0', carrier='AA', day=1, dep_delay=4, dep_time=1824, dest='LAX', d

In [9]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [10]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1509, dest='MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin='JFK', tailnum='N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled='0', carrier='AA', day=1, dep_delay=7, dep_time=917, dest='MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin='JFK', tailnum='N5DWAA', year=2014)]

### Count the flights that departed early and arrived late

In [11]:
flightsDF.where("dep_delay < 0 AND arr_delay > 0").count()

30239

### Find the flightwith the longest arrival delay

In [12]:
sqlContext.sql("select * from flight order by arr_delay desc").take(1)

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled='0', carrier='AA', day=4, dep_delay=1498, dep_time=727, dest='DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin='EWR', tailnum='N4WJAA', year=2014)]

In [13]:
flightsDF.sort(flightsDF.arr_delay.desc()).take(1)

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled='0', carrier='AA', day=4, dep_delay=1498, dep_time=727, dest='DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin='EWR', tailnum='N4WJAA', year=2014)]

### Find the top 10 destinations ordered by the number of flights

In [14]:
sqlContext.sql("select dest, count(1) as number from flight group by dest order by number desc").take(10)

[Row(dest='LAX', number=14434),
 Row(dest='ATL', number=12808),
 Row(dest='SFO', number=11907),
 Row(dest='MCO', number=11709),
 Row(dest='BOS', number=11609),
 Row(dest='ORD', number=11589),
 Row(dest='MIA', number=9928),
 Row(dest='CLT', number=9624),
 Row(dest='FLL', number=9471),
 Row(dest='DCA', number=6748)]

In [15]:
from pyspark.sql.functions import *
flightsDF.groupBy('dest').count().orderBy(desc('count')).take(10)

[Row(dest='LAX', count=14434),
 Row(dest='ATL', count=12808),
 Row(dest='SFO', count=11907),
 Row(dest='MCO', count=11709),
 Row(dest='BOS', count=11609),
 Row(dest='ORD', count=11589),
 Row(dest='MIA', count=9928),
 Row(dest='CLT', count=9624),
 Row(dest='FLL', count=9471),
 Row(dest='DCA', count=6748)]

### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

In [16]:
sqlContext.sql("select dest, avg(arr_delay) as avg_delay from flight where arr_delay > 0 group by dest order by avg_delay desc").take(10)

[Row(dest='EGE', avg_delay=80.59016393442623),
 Row(dest='AVP', avg_delay=67.0),
 Row(dest='JAC', avg_delay=59.666666666666664),
 Row(dest='CAK', avg_delay=57.907042253521126),
 Row(dest='TUL', avg_delay=56.99152542372882),
 Row(dest='MSN', avg_delay=54.80237154150198),
 Row(dest='IAD', avg_delay=53.324599708879184),
 Row(dest='BGR', avg_delay=52.73504273504273),
 Row(dest='OKC', avg_delay=52.14503816793893),
 Row(dest='TVC', avg_delay=51.15151515151515)]

In [17]:
flightsDF.where('arr_delay > 0').groupBy('dest').avg('arr_delay').orderBy(desc('avg(arr_delay)')).take(10)

[Row(dest='EGE', avg(arr_delay)=80.59016393442623),
 Row(dest='AVP', avg(arr_delay)=67.0),
 Row(dest='JAC', avg(arr_delay)=59.666666666666664),
 Row(dest='CAK', avg(arr_delay)=57.907042253521126),
 Row(dest='TUL', avg(arr_delay)=56.99152542372882),
 Row(dest='MSN', avg(arr_delay)=54.80237154150198),
 Row(dest='IAD', avg(arr_delay)=53.324599708879184),
 Row(dest='BGR', avg(arr_delay)=52.73504273504273),
 Row(dest='OKC', avg(arr_delay)=52.14503816793893),
 Row(dest='TVC', avg(arr_delay)=51.15151515151515)]

### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

In [18]:
flightsDF.sample(False,0.01).registerTempTable("sample_flight")
sqlContext.sql("select avg(dep_delay) from sample_flight").collect()

[Row(avg(dep_delay)=13.81475604918683)]

In [19]:
flightsDF.sample(False,0.01).groupBy().avg('dep_delay').collect()

[Row(avg(dep_delay)=12.98184627269216)]

### For all flights from JFK during June, show the average departure delay for each destination

In [20]:
sqlContext.sql("select dest, avg(dep_delay) as delay from flight where origin='JFK' and month=6 group by dest order by delay desc").collect()

[Row(dest='MSP', delay=28.69047619047619),
 Row(dest='SMF', delay=22.96551724137931),
 Row(dest='OAK', delay=22.75862068965517),
 Row(dest='DEN', delay=19.083333333333332),
 Row(dest='JAX', delay=18.49425287356322),
 Row(dest='CVG', delay=17.571428571428573),
 Row(dest='SAT', delay=17.17241379310345),
 Row(dest='PSE', delay=17.133333333333333),
 Row(dest='BUR', delay=16.428571428571427),
 Row(dest='SLC', delay=14.965909090909092),
 Row(dest='RSW', delay=14.913793103448276),
 Row(dest='BNA', delay=14.172413793103448),
 Row(dest='ATL', delay=13.491620111731844),
 Row(dest='IAD', delay=13.333333333333334),
 Row(dest='BOS', delay=13.006864988558352),
 Row(dest='BUF', delay=12.900826446280991),
 Row(dest='SAN', delay=12.286713286713287),
 Row(dest='SJC', delay=12.241379310344827),
 Row(dest='ORD', delay=12.136054421768707),
 Row(dest='CLT', delay=12.104803493449781),
 Row(dest='CMH', delay=11.862068965517242),
 Row(dest='SYR', delay=11.656862745098039),
 Row(dest='SEA', delay=11.63902439024

In [21]:
flightsDF.where("origin = 'JFK' and month = 6").groupBy('dest').avg('dep_delay').orderBy(desc('avg(dep_delay)')).collect()

[Row(dest='MSP', avg(dep_delay)=28.69047619047619),
 Row(dest='SMF', avg(dep_delay)=22.96551724137931),
 Row(dest='OAK', avg(dep_delay)=22.75862068965517),
 Row(dest='DEN', avg(dep_delay)=19.083333333333332),
 Row(dest='JAX', avg(dep_delay)=18.49425287356322),
 Row(dest='CVG', avg(dep_delay)=17.571428571428573),
 Row(dest='SAT', avg(dep_delay)=17.17241379310345),
 Row(dest='PSE', avg(dep_delay)=17.133333333333333),
 Row(dest='BUR', avg(dep_delay)=16.428571428571427),
 Row(dest='SLC', avg(dep_delay)=14.965909090909092),
 Row(dest='RSW', avg(dep_delay)=14.913793103448276),
 Row(dest='BNA', avg(dep_delay)=14.172413793103448),
 Row(dest='ATL', avg(dep_delay)=13.491620111731844),
 Row(dest='IAD', avg(dep_delay)=13.333333333333334),
 Row(dest='BOS', avg(dep_delay)=13.006864988558352),
 Row(dest='BUF', avg(dep_delay)=12.900826446280991),
 Row(dest='SAN', avg(dep_delay)=12.286713286713287),
 Row(dest='SJC', avg(dep_delay)=12.241379310344827),
 Row(dest='ORD', avg(dep_delay)=12.136054421768707)

### For every origin/dest pair, count the number of flights

In [22]:
sqlContext.sql("select origin, dest, count(1) as number from flight group by origin, dest order by origin, number desc").collect()

[Row(origin='EWR', dest='SFO', number=4539),
 Row(origin='EWR', dest='BOS', number=4268),
 Row(origin='EWR', dest='LAX', number=4226),
 Row(origin='EWR', dest='ATL', number=4182),
 Row(origin='EWR', dest='MCO', number=4164),
 Row(origin='EWR', dest='CLT', number=3921),
 Row(origin='EWR', dest='ORD', number=3272),
 Row(origin='EWR', dest='IAH', number=3073),
 Row(origin='EWR', dest='FLL', number=2909),
 Row(origin='EWR', dest='DEN', number=2243),
 Row(origin='EWR', dest='DFW', number=2224),
 Row(origin='EWR', dest='DTW', number=2149),
 Row(origin='EWR', dest='MIA', number=2094),
 Row(origin='EWR', dest='PHX', number=2002),
 Row(origin='EWR', dest='PBI', number=1874),
 Row(origin='EWR', dest='TPA', number=1833),
 Row(origin='EWR', dest='BNA', number=1797),
 Row(origin='EWR', dest='STL', number=1740),
 Row(origin='EWR', dest='LAS', number=1555),
 Row(origin='EWR', dest='MDW', number=1537),
 Row(origin='EWR', dest='MSP', number=1485),
 Row(origin='EWR', dest='SEA', number=1364),
 Row(origi

In [23]:
flightsDF.groupBy(['origin','dest']).count().orderBy(['origin','count'],ascending=[1,0]).collect()

[Row(origin='EWR', dest='SFO', count=4539),
 Row(origin='EWR', dest='BOS', count=4268),
 Row(origin='EWR', dest='LAX', count=4226),
 Row(origin='EWR', dest='ATL', count=4182),
 Row(origin='EWR', dest='MCO', count=4164),
 Row(origin='EWR', dest='CLT', count=3921),
 Row(origin='EWR', dest='ORD', count=3272),
 Row(origin='EWR', dest='IAH', count=3073),
 Row(origin='EWR', dest='FLL', count=2909),
 Row(origin='EWR', dest='DEN', count=2243),
 Row(origin='EWR', dest='DFW', count=2224),
 Row(origin='EWR', dest='DTW', count=2149),
 Row(origin='EWR', dest='MIA', count=2094),
 Row(origin='EWR', dest='PHX', count=2002),
 Row(origin='EWR', dest='PBI', count=1874),
 Row(origin='EWR', dest='TPA', count=1833),
 Row(origin='EWR', dest='BNA', count=1797),
 Row(origin='EWR', dest='STL', count=1740),
 Row(origin='EWR', dest='LAS', count=1555),
 Row(origin='EWR', dest='MDW', count=1537),
 Row(origin='EWR', dest='MSP', count=1485),
 Row(origin='EWR', dest='SEA', count=1364),
 Row(origin='EWR', dest='DCA', c

In [24]:
sc.stop()